In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

# Indexing document

In [2]:
import minsearch, json

/Users/air/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
with open('documents.json', 'rt') as input_file:
    docs_raw = json.load(input_file)

In [ ]:
docs_raw[:4]

In [ ]:
docs_raw[0]['documents']

In [4]:
documents = []

# put the course name into each document
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0:1]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'}]

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [7]:
question = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

# get the results of query from the minsearch index

In [9]:
results = index.search(
    question,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict={'question': 3.0, 'section': 0.5},
    num_results=5
)

print(results)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}, {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. Th

# Use mistral al for generation using the retreived-augmented prompt 

In [ ]:
! pip install mistralai

In [10]:
import mistralai
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


True

In [11]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

api_key = os.getenv('MISTRAL_API_KEY')
client = MistralClient(api_key=api_key)

In [12]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [13]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [14]:
def llm(prompt):
    response = client.chat(
        model='open-mistral-7b',
        messages=[
            ChatMessage(role="user", content=prompt)
        ]
    )

    return response.choices[0].message.content

In [15]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [16]:
response = rag(query)

print(response)

To run Kafka, the answer depends on whether you're using Java or Python.

For Java, you can run a Kafka producer or consumer by navigating to the project directory in your terminal and running:

```
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

Replace `<jar_name>` with the name of your Java project's JAR file.

For Python, first, create a virtual environment and install the required packages:

```
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
```

Then, activate the virtual environment and run your Python Kafka script.

If you're encountering a "Permission denied" error when running `./build.sh` for Python, you can fix it by running:

```
chmod +x build.sh
```

Finally, if you're using Python and encountering a "ModuleNotFoundError: No module named 'kafka.vendor.six.moves'" error, consider using `kafka-python-ng` instead of the standard `kafka-python` package. You can install it with:

```
pip insta

In [33]:
rag('the course has already started, can I still enroll?')

"Based on the provided context, the course has already started on January 15, 2024, at 17:00. However, the FAQ mentions that even if you haven't registered yet, you're still eligible to submit the homeworks. Just be aware of the deadlines for turning in the final projects. The course materials will also be kept available after it finishes, allowing you to follow the course at your own pace. You can also continue looking at the homeworks and preparing for the next cohort. Before the course started, you were advised to install and set up all the dependencies and requirements. If you're taking the course in self-paced mode, support is available through the Slack channel."

In [34]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [17]:
from elasticsearch import Elasticsearch

In [18]:
es_client = Elasticsearch('http://localhost:9200') 

In [19]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [20]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [21]:
from tqdm.auto import tqdm

In [22]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [23]:
query = 'I just disovered the course. Can I still join it?'

In [24]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [25]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [26]:
rag(query)

"Based on the context provided, yes, you can still join the course even if it has already started. You're still eligible to submit homeworks, but be aware of the deadlines for turning in final projects. The course materials will also be kept available after the course finishes, allowing you to follow the course at your own pace. Before the course starts, you can prepare by installing and setting up all the dependencies and requirements. You can also seek support through the Slack channel during the self-paced mode."